In [1]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from PIL import Image

import torch
import yaml


Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CPU (Intel Xeon E5-2660 v4 2.00GHz)
Setup complete ✅ (28 CPUs, 251.2 GB RAM, 0.4/125.6 GB disk)


In [2]:
device = "0" if torch.cuda.is_available() else "cpu"
if device == "0":
    torch.cuda.set_device(0) # Set to your desired GPU number

print(device)

cpu


In [3]:
# Create a new YOLO model from scratch
#model = YOLO('yolov8n.yaml')

# Load a pretrained YOLO model (recommended for training) - for now, we keep the small version
model = YOLO('yolov8n.pt')

In [4]:
# TRAIN the model on our dataset (data.yml config file) for 10 epochs
results = model.train(
   data='data.yaml',
   #imgsz=480,  # we are trying with several img size so we do not precise the size -> will automatically resize all images to 640x640
   epochs=50,
   batch=32,
   name='yolov8n_custom')

New https://pypi.org/project/ultralytics/8.0.203 available 😃 Update with 'pip install -U ultralytics'


Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CPU (Intel Xeon E5-2660 v4 2.00GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_custom16, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, opt

THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_test_2datasets/train
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_test_2datasets/train


train: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_test_2datasets/train/labels.cache... 1180 images, 452 backgrounds, 0 corrupt: 100%|██████████| 1632/1632 [00:00<?, ?it/s]
/home/eec42/.conda/envs/pdm/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_test_2datasets/val
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_test_2datasets/val


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_test_2datasets/val/labels.cache... 166 images, 78 backgrounds, 0 corrupt: 100%|██████████| 244/244 [00:00<?, ?it/s]
Plotting labels to runs/detect/yolov8n_custom16/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/yolov8n_custom16
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
  0%|          | 0/51 [00:00<?, ?it/s]

: 

## **Validation on test subset**

In [5]:
# EVALUATE the model's performance on the test set
results = model.val(split='test', save_json=True)

# Export the model to ONNX format
#success = model.export(format='onnx')

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)


Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_experiment_no_backgd/test/uav_thermal_waterfowl/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_experiment_no_backgd/test/uav_thermal_waterfowl/images


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_experiment_no_backgd/test/uav_thermal_waterfowl/labels.cache... 70 images, 0 backgrounds, 0 corrupt: 100%|██████████| 70/70 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.31it/s]
                   all         70       1479      0.477      0.761      0.632       0.29
Speed: 0.2ms preprocess, 6.5ms inference, 0.0ms loss, 1.8ms postprocess per image
Saving runs/detect/yolov8n_custom142/predictions.json...
Results saved to runs/detect/yolov8n_custom142


In [ ]:
# Perform MODEL PREDICTION on the test set
model.predict(
    model = 'runs/detect/yolov8n_custom93',
    source ='/gpfs/gibbs/project/jetz/eec42/data/baseline1/test/images',
    conf = 0.25
)

In [ ]:
# Save PREDICTIONS PREVIEWS

import visualization_utils as visutils
import os

#results = model(source = '/gpfs/gibbs/project/jetz/eec42/data/baseline1/test/images/')
threshold = 0.5
category_id_to_name = {0: 'bird'}
detection_boxes = []

for result in results:
    im_array = result.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    #im.show()  # show image
    im.save('/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/yolov8n_custom12/predictions/' + os.path.basename(result.path).split('.png')[0] + '.jpg')  # save image


## **Validation on test subset per dataset**

In [3]:
database1_source = ['global_birds_poland', 'global_birds_palmyra', 'global_birds_penguins',
                    'global_birds_mckellar', 'global_birds_newmexico', 
                    'global_birds_pfeifer', 'uav_thermal_waterfowl']

In [11]:
model = YOLO('runs/detect/yolov8n_custom14/weights/best.pt')

In [10]:
for dataset in database1_source:

    # Change test folder
    fname = "data.yaml"
    stream = open(fname, 'r')
    data = yaml.safe_load(stream)
    data['test'] = 'test/' + dataset + '/images/'
    with open(fname, 'w') as yaml_file:
        yaml_file.write( yaml.dump(data, default_flow_style=False))
    
    model.val(model = 'runs/detect/yolov8n_custom14', split='test', save_json=True, )


AttributeError: 'str' object has no attribute 'val'